# Семантический поиск на основе transformers

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.8 MB/s eta 0:00:00


Google drive подключение

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Загружаем датасет

In [39]:
from datasets import load_dataset, DatasetDict

atom_dataset = load_dataset('csv', data_files={
    "train": '/content/answer_ds.csv',
})

Создадим новый датасет

In [40]:
new_dataset_dict = DatasetDict({'train': atom_dataset["train"]})

In [41]:
new_dataset_dict['train'] = atom_dataset['train']
# new_dataset_dict['train'] = new_dataset_dict['train']#.unique('video_id')
new_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['query', 'answer'],
        num_rows: 50
    })
})

Оставим только необходимые столбцы

In [42]:
columns = new_dataset_dict["train"].column_names
columns_to_keep = ["query", "answer"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
new_dataset = new_dataset_dict["train"].remove_columns(columns_to_remove)
new_dataset

Dataset({
    features: ['query', 'answer'],
    num_rows: 50
})

Датасет для создания эмбендинга

In [43]:
def concatenate_text(x):
    return {
        "text":x["query"]
        + " \n "
        + x["answer"]
    }


new_atom_dataset = new_dataset.map(concatenate_text)

Загружаем модель https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-v1

In [44]:
from transformers import AutoTokenizer, AutoModel


model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [45]:
import torch

device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [46]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [47]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [48]:
embedding = get_embeddings(new_atom_dataset["text"][0])
new_atom_dataset["text"][1]
# embedding.shape

'Какие закупки согласно ЕОСЗ являются конкурентными? \n 1. Конкурентная закупка в электронной форме, участниками которой с учетом особенностей, установленных Правительством Российской Федерации в соответствии с пунктом 2 части 8 статьи 3 настоящего Федерального закона, могут быть только субъекты малого и среднего предпринимательства (далее также - конкурентная закупка с участием субъектов малого и среднего предпринимательства), осуществляется в соответствии со статьями 3.2 и 3.3 настоящего Федерального закона и с учетом требований, предусмотренных настоящей статьей.2. Конкурентная закупка с участием субъектов малого и среднего предпринимательства осуществляется путем проведения конкурса в электронной форме, аукциона в электронной форме, запроса котировок в электронной форме или запроса предложений в электронной форме.'

In [49]:
embeddings_dataset = new_atom_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

In [50]:
from huggingface_hub import notebook_login

notebook_login()

In [51]:
embeddings_dataset.push_to_hub("Denis431/atomic_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [52]:
embeddings_dataset

Dataset({
    features: ['query', 'answer', 'text', 'embeddings'],
    num_rows: 50
})

In [53]:
from datasets import load_dataset

automarkup_dataset = load_dataset('csv', data_files={
    "test": '/content/answer_ds.csv',
})

In [54]:
automarkup_dataset['test'][10]

{'query': 'Кто может давать официальные разъяснения ЕОСЗ и распорядительных документов Росатома в области закупок?',
 'answer': 'Методолог Корпорации'}

In [55]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['query', 'answer', 'text', 'embeddings'],
    num_rows: 50
})

Оценка на основании torch score

In [22]:
from transformers import AutoTokenizer, AutoModel
import torch

# CLS Pooling - Take output from first token
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

# Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = cls_pooling(model_output)

    return embeddings

# Load model and tokenizer from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-mpnet-base-dot-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-mpnet-base-dot-v1")

# Sentences we want sentence embeddings for
query = "На какой срок утверждается ГПЗ?"
docs = embeddings_dataset["answer"]
doc_emb = embeddings_dataset["embeddings"]

# Encode query and docs
query_emb = encode(query)

doc_emb_tensor = torch.tensor(doc_emb)  # Convert doc_emb to a tensor
scores = torch.mm(query_emb, doc_emb_tensor.transpose(0, 1))[0].cpu().tolist()

# Combine docs, scores, and video_ids
doc_score_pairs = list(zip(embeddings_dataset["query"], docs, scores))

# Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[2], reverse=True)

# Output passages, scores, and video_ids
for video_id, doc, score in doc_score_pairs[:10]:
    print(video_id, score, doc)



На какой срок утверждается ГПЗ? 29.79566764831543 на 1 год
Какие способы закупки являются торгами для заказчиков 2 группы? 22.77951431274414 К второй группе конкурентному способу закупок, только конкурс; 
Что должен содержать запрос ТКП? 22.315454483032227 условия исполнения договора по совокупности критериев, установленных в документации 
Может ли председатель закупочной комиссии воздержаться от голосования при проведении заседания закупочной комисии? 22.215118408203125 не может
В результате подготовки к закупке стоимость продукции увеличилась на 5%. Обязан ли заказчик в соответствии ЕОСЗ осуществить корректировку ГПЗ? 22.03170394897461 нет. при изменении НМЦ более чем на 10% по сравнению с плановой стоимостью закупки, сделанном в процессе подготовки и утверждения извещения и документации о закупке вносится корректировка
В каких случаях можно внести изменения в ГПЗ? 21.89141845703125 В течение календарного года возможны корректировки и изменения утвержденной ГПЗ в связи с :а) корректи

# Поиск на основе мультиязычной модели https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v1

In [25]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d80fc73bf8725fe945aa5400943811de60bb445dfbacb9befb0b1f311f9e3106
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [26]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.45k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

Оценка на основании pytorch_cos_sim

In [29]:
from sentence_transformers import SentenceTransformer, util
import torch


# Предложения, для которых мы хотим получить векторные представления
query = "На какой срок утверждается ГПЗ?"
docs = embeddings_dataset["answer"]


# Кодируем запрос и предложения
query_emb = model.encode(query, convert_to_tensor=True)
docs_emb = model.encode(docs, convert_to_tensor=True)

# Вычисляем косинусное сходство между запросом и всеми векторными представлениями предложений
scores = util.pytorch_cos_sim(query_emb, docs_emb)[0].cpu().tolist()

# Соединяем предложения, оценки и идентификаторы видео
doc_score_pairs = list(zip(embeddings_dataset["query"], docs, scores))

# Сортируем по убыванию оценки
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[2], reverse=True)

# Выводим предложения, оценки и идентификаторы видео
for idx, (video_id, doc, score) in enumerate(doc_score_pairs[:10]):
    print(f"{idx + 1}. Answer: {doc}, Score: {score}")


1. Answer: В течение календарного года возможны корректировки и изменения утвержденной ГПЗ в связи с :а) корректировками бюджета, инвестиционной, производственной и иных программ и мероприятий заказчика, если данные корректировки меняют сведения, указанные в ГПЗ; б) изменением предмета, способа, формы закупки, сроков приобретения продукции и исполнения договора, указанных в ГПЗ, в том числе при проведении повторной закупки;в) при изменении НМЦ более чем на 10% по сравнению с плановой стоимостью закупки, сделанном в процессе подготовки и утверждения извещения и документации о закупке., Score: 0.2552986145019531
2. Answer: в течение 3 рабочих дней после размещения на официальном сайте протокола заседания комиссии об уточнении заявок, Score: 0.22391609847545624
3. Answer: на 1 год, Score: 0.21749912202358246
4. Answer: В случае, если по окончании срока подачи заявок на участие в конкурентной закупке подана только одна заявка или не подано ни одной такой заявки, конкурентная закупка призна